In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from clawpack import pyclaw
from ipywidgets import interact

# Working with mapped grids in PyClaw

## Helpful resources

The relevant concepts and ideas are explained in Chapter 23 of [LeVeque's book](https://www.clawpack.org/fvmhp_materials/).  This notebook is primarily focused on implementation details.

- [Code and animations from 2008 SIAM Review article](https://faculty.washington.edu/rjl/pubs/circles/) -- uses Clawpack 4.x, not current version (and not PyClaw)
- [Tutorial on geometry objects in PyClaw](https://github.com/clawpack/apps/blob/master/notebooks/pyclaw/pyclaw_geometry.ipynb)
- The following PyClaw examples use mapped grids:
    - `pyclaw/examples/acoustics_2d_mapped`
    - `pyclaw/examples/advection_2d_annulus`
    - `pyclaw/examples/shallow_sphere`
- Riemann solvers that work on mapped grids (all are in `clawpack/riemann/src`):
    - `rpn2_acoustics_mapped.f90`
    - `rpn2_euler_mapgrid.f90`
    - `rpn3_euler_mapgrid.f90`

## Examples

Here are a couple of examples of mapped grids (mapping to a circle) from the SIREV article mentioned above.

In [ ]:
xi  = pyclaw.Dimension(-1.,1.,30,name='xi')
eta = pyclaw.Dimension(-1.,1.,30,name='eta')
grid = pyclaw.geometry.Grid((xi,eta))

In [ ]:
def square2circle(xi,eta,r1=1.0):
    # radial projection mapping from Section 3.1
    d = np.maximum(np.abs(xi),np.abs(eta))
    r = np.sqrt(xi**2 + eta**2)
    r = np.maximum(r, 1.e-10)
    xp = r1 * d * xi/r
    yp = r1 * d * eta/r
    return [xp, yp]

In [ ]:
grid.mapc2p = square2circle
grid.plot(num_ghost=1);

In [ ]:
def square2circle_smooth(xi,eta,r1=1.0):
    # from Section 3.2
    d = np.maximum(np.abs(xi),np.abs(eta))
    d = np.maximum(d, 1e-10)

    D = r1 * d/np.sqrt(2)
    R = r1 * d
    #R = r1 * np.ones_like(d)# * d

    center = D - np.sqrt(R**2 - D**2)
    xp = D/d * np.abs(xi)
    yp = D/d * np.abs(eta)

    ij = np.nonzero(np.abs(eta)>=np.abs(xi))
    yp[ij] = center[ij] + np.sqrt(R[ij]**2 - xp[ij]**2)
    ij = np.nonzero(np.abs(xi)>=np.abs(eta))
    xp[ij] = center[ij] + np.sqrt(R[ij]**2 - yp[ij]**2)

    xp = np.sign(xi) * xp
    yp = np.sign(eta) * yp
    
    return [xp, yp]

In [ ]:
grid.mapc2p = square2circle_smooth
grid.plot(num_ghost=1);

# Extra steps required to set up a problem on a mapped grid

Compared to setting up a problem on a uniform cartesian grid, a little extra work is required:

1. Define the mapping function
2. Compute geometry data and store in `aux`
3. Define the Riemann solver to work on mapped grids
4. Plot the data on a mapped grid

Let's go through these steps, following the example in `pyclaw/examples/acoustics_2d_mapped`.

## Defining the mapping function

For this problem, the mapping involves two cylinders embedded in a larger region.  The mapping function is defined in
`pyclaw/examples/acoustics_2d_mapped/acoustics_2d_inclusions.py`, in the function `inclusion_mapping()`.  This function is not used directly by the simulation code, but will be used to compute the required geometric data and for plotting.  It takes the reference, or computational, coordinates as input and returns the physical coordinates.

## Computing and storing geometry data

The quantities required in the code are:
 - The normal vector to each face
 - The ratio of each edge length to the reference edge length
 - The ratio of each cell area or volume to the reference cell area or volume

These quantities are computed in `pyclaw/examples/acoustics_2d_mapped/acoustics_2d_inclusions.py` in the function `compute_geometry()`.

By convention, the geometry data is stored in the following way in 2D:

```
    state.aux[0,:,:] = a_x
    state.aux[1,:,:] = a_y
    state.aux[2,:,:] = length_left
    state.aux[3,:,:] = b_x
    state.aux[4,:,:] = b_y
    state.aux[5,:,:] = length_bottom
    state.aux[6,:,:] = area
    state.index_capa = 6
```

Here $a_x, a_y, b_x, b_y$ are the components of the normal vector to the face at the left (or bottom) edge of each cell.  Meanwhile `length_left` and `length_bottom` are the side length ratios (for the left and bottom edges) of the physical grid to the reference grid.

## Riemann solvers on mapped grids

It's a good idea to look at existing mapped-grid Riemann solvers in order to understand what must be done.
There is a list of existing solvers in the resources section above.

The main things that need to be kept in mind are:

1. One needs to use the appropriate eigenvectors, corresponding in the linear case to those of the matrix $n^x A + n^y B$.  For many systems, this is equivalent to finding the normal and tangential velocity components, and then using the eigenvectors $A$ or $B$ to decompose the jump in $q$.
2. The contribution from 
The approach used in those solvers is to locally rotate the data, solve the Riemann problem in the reference coordinates, and then rotate the data back.  This rotation only needs to be performed for data with a geometric meaning (usually this is a velocity vector).

We will follow how this is done for 2D acoustics.  First, depending on whether we are solving a Riemann problem for an $x$ or $y$ face (in the reference grid), we retrieve the appropriate face normal vector and length ratio:

```
    if (ixy.eq.1) then
        inx = 1
        iny = 2
        ilenrat = 3
    else
        inx = 4
        iny = 5
        ilenrat = 6
    endif

! Rotation matrix:
!               [ alpha  beta ]
!               [-beta  alpha ]

! Determine normal velocity components at this edge:
    do i=2-mbc,mx+mbc
        alpha = auxl(inx,i)
        beta  = auxl(iny,i)
        unorl = alpha*ql(2,i) + beta*ql(3,i)
        unorr = alpha*qr(2,i-1) + beta*qr(3,i-1)
```

The next section of the code is identical to the standard 2D acoustics Riemann solver, except that it uses the
normal and tangential velocity components just computed:

```
        delta(1) = ql(1,i) - qr(1,i-1)
        delta(2) = unorl - unorr

        zi  = auxl(8,i)
        zim = auxl(8,i-1)
        ci  = auxl(9,i)
        cim = auxl(9,i-1)

        a1 = (-delta(1) + zi*delta(2)) / (zim + zi)
        a2 =  (delta(1) + zim*delta(2)) / (zim + zi)
```

Finally, in the computation of the waves (and fluctuations), we need to use the eigenvectors of the matrix $n^x A + n^y B$,
corresponding to the local tangential and orthogonal directions relative to the cell face:

```
        wave(1,1,i) = -a1*zim
        wave(2,1,i) = a1 * alpha
        wave(3,1,i) = a1 * beta
        s(1,i) = -cim * auxl(ilenrat,i)

        wave(1,2,i) = a2*zi
        wave(2,2,i) = a2 * alpha
        wave(3,2,i) = a2 * beta
        s(2,i) = ci * auxl(ilenrat,i)
```
Notice that we also rescale the wave speeds based on the ratio of the edge length in the physical domain to that in the reference domain.

Finally, note that the contributions to each cell average that are returned from the Riemann solver need to be rescaled
by the cell area (or volume) ratio.  But this is handled automatically in the Clawpack code outside the Riemann solver, as long as the appropriate values have been set, as described above.

## Plotting data on a mapped grid

The easiest approach to plotting solutions on a mapped grid is to use VisClaw.  In that case, you only need to let the `setplot` function know about your mapping:

```
def setplot(plotdata):
    plotdata.mapc2p = my_mapping
    ...
```

This is the approach used in the 2D acoustics example.